<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%202/NLP_07_extraccio%CC%81n_de_caracteri%CC%81sticas_Big_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracción de características en grandes volúmenes de datos
Para extraer las características de un corpus muy grande de textos, conviene procesarlo mediante objetos *stream*. Así no se carga todo el corpus en memoria sino que se analiza documento a documento.

## Conjunto de textos de "mundocine"
Vamos a utilizar un conjunto de datos formado por una serie de críticas de películas de cine, almacenadas en formato XML en el directorio `'criticas'` (una crítica por archivo). Hemos preparado una función de tipo `generator` que procesa el directorio donde están los archivos de las críticas y devuelve por cada archivo XML una tupla con 4 valores:
 - Nombre de la película (*string*)
 - Resumen breve de la crítica (*string*)
 - Texto de la crítica (*string*)
 - Valoración de la película (*int* de 1 a 5)

In [ ]:
import os, re
from xml.dom.minidom import parseString

def parse_folder(path):
    """generator that reads the contents of XML files in a folder
    Returns the <body> of the <review> in each XML file.
    XML files encoded as 'latin-1'"""
    for file in sorted([f for f in os.listdir(path) if f.endswith('.xml')],
                        key=lambda x: int(re.match(r'\d+',x).group())):
        with open(os.path.join(path, file), encoding='latin-1') as f:
            doc=parseString(f.read())

            titulo = doc.documentElement.attributes["title"].value

            btxt = ""
            review_bod = doc.getElementsByTagName("body")
            if len(review_bod) > 0:
                for node in review_bod[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        btxt += node.data + " "

            rtxt = ""
            review_summ = doc.getElementsByTagName("summary")
            if len(review_summ) > 0:
                for node in review_summ[0].childNodes:
                    if node.nodeType == node.TEXT_NODE:
                        rtxt += node.data + " "
                        
            rank = int(doc.documentElement.attributes["rank"].value)
            
            yield titulo, rtxt, btxt, rank


### Ejercicio 1
Carga la primera crítica del directorio usando el método `next` sobre la función `parse_folder` en el objeto `critica`. Muestra sus 4 valores.

## Extracción de características básicas
Utilizando la librería `textaCy` procesamos cada documento y generamos una serie de estadísticos. \
Guardaremos los resultados en un objeto `DataFrame` de Pandas.\
Como característica de cada crítica vamos a extraer:
- Título de la película
- Longitud (en caracteres) del resumen
- Longitud (en caracteres) del texto de la crítica
- Puntuación de la crítica

### Ejercicio 2
Completa el código siguiente para generar el `DataFrame`

In [ ]:
import pandas as pd

#creamos una lista en blanco
datos = 

#recorremos las críticas y calculamos sus métricas
for c in parse_folder("criticas"):
    datos.append({
        'título': ___,
        'LongResumen': ___,
        'LongCritica': ___,
        'puntuación': ___
    })

resumen = pd.DataFrame(datos)

In [ ]:
resumen

## Limpieza de texto
Para poder extraer las características vectoriales del corpus es conveniente realizar primero una limpieza y pre-procedado de cada documento.\
Vamos a suponer que queremos preparar este conjunto de textos para entrenar un modelo que prediga la puntuación de cada crítica a partir del texto de la crítica.\
Realizaremos el siguiente procesado:
- Introducir un espacio después de determinados signos de puntuación (".", "?") para que el tokenizado sea correcto
- Separar el texto en *tokens*
- Eliminar los *tokens* de tipo *stop-word*, signos de puntuación o espacios o de longitud 1
- Convertir las entidades de tipo `PER` al token *persona*
- Lematizar el texto y pasarlo a minúsculas

In [ ]:
import spacy

nlp = spacy.load("es_core_news_sm")

def normaliza(texto):
    #separamos después de ciertos signos de puntuación
    texto = re.sub(r"([\.\?])", r"\1 ", texto)
    doc = nlp(texto)
    tokens = [t for t in doc if not t.is_punct and not t.is_stop and not t.is_space and len(t.text)>1]
    palabras = []
    for t in tokens:
        if t.ent_iob_=='B' and t.ent_type_=='PER':
            palabras.append('persona')
        elif t.ent_iob_=='I' and t.ent_type_=='PER':
            continue
        else:
            palabras.append(t.lemma_.lower())
    salida = ' '.join(palabras)
    
    return salida

### Ejercicio 3
Comprueba su funcionamiento en la crítica previamente descargada (variable `critica`)

### Análisis morfológico
En una crítica tiene mucha importancia los adjetivos utilizados.\
Crea una función para filtrar sólo los adjetivos utilizados en cada crítica (utiliza el lema de cada adjetivo).

In [ ]:
def extraer_adj(texto):
    texto = re.sub(r"([\.\?])", r"\1 ", texto)
    doc = nlp(texto)
    tokens = [t.lemma_ for t in doc if t.pos_=="ADJ"]
    
    return ' '.join(tokens)

Comprobamos su funcionamiento en la crítica previamente descargada (variable `critica`)

In [ ]:
extraer_adj(critica[2])

## Extracción de características *sparse* con `scikit-learn`
Vamos a calculas las matrices de características *bag-of-words* y *tfidf* del conjunto de textos anterior.\
Vamos a usar la librería `scikit-learn` para vectorizar los documentos.

In [ ]:
#Para no tener que cargar todas las críticas en memoria,
#creamos un generador que devuelve iterativamente el
#texto procesado de cada crítica

def generaCritica(criticas):
    """Función de tipo generator que devuelve el
    texto normalizado de cada crítica.
    Entrada:
    criticas: objeto 'parse_folder' que itera
    sobre el directio de las críticas
    Salida:
    texto normalizado de cada crítica"""
    for c in criticas:
        yield normaliza(c[2])

Comprobamos su funcionamiento generando el texto normalizado de la primera crítica

In [ ]:
next(generaCritica(parse_folder("criticas")))

Vectorizamos todo el conjunto de datos usando las funciones de `scikit-learn`.\
Estas funciones admiten un objeto `generator` como argumento de entrada.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer()

criticas = generaCritica(parse_folder("criticas"))
BoW_criticas = vect.fit_transform(criticas)
BoW_criticas

### Ejercicio 4
Genera distintas variantes de matrices de características para el conjunto de las críticas. Prueba con:
- Matriz TF-IDF
- Matriz BoW con unigramas y bigramas
- Matriz TF-IDF eliminando las palabras menos frecuentes y las más frecuentes (mínimo de 2 y máximo de 5 documentos)
- Muestra cuáles son las palabras más frecuentes eliminadas

In [ ]:
#matriz TF-IDF


In [ ]:
#BoW de unigramas y bigramas


In [ ]:
#Matriz TF-IDF eliminando las palabras menos frecuentes y las más frecuentes (mínimo de 2 y máximo de 5 documentos)


In [ ]:
#Palabras más frecuencias eliminadas (atributo 'stop_words_' del vectorizador)


## Extracción de Word embeddings
Ahora vamos a calcular los *word vectors* de las palabras de nuestro conjunto de datos, usando la clase `word2vec` de la librería `gensim`.\
Esta librería acepta como argumento de entrada un objeto `iterador` que generará el texto pre-procesado de la siguiente crítica en la secuencia.\
Vamos a usar las funciones de pre-procesado de la librería `gensim`.\
Primero definimos un objeto de tipo `iterator` para recorrer las críticas. Se diferencia de un simple `generator` en que se puede reiniciar la generación de la secuencia (necesario para el modelo `word2vec`)

In [ ]:
from gensim.utils import simple_preprocess
        
class PreprocesaCriticas(object):
    """Pre-procesa el corpus de críticas con la función 'simple_preprocess'
    de la librería gensim
    Entrada: directorio de críticas
    Salida: iterador sobre las críticas (como lista de tokens)"""
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        for c in parse_folder(self.dirname):
            yield normaliza(c[2]).split(' ')

In [ ]:
#instanciamos un objeto para nuestro directorio
criticas = PreprocesaCriticas("criticas")

Para probar su funcionamiento con la primera crítica lo convertimos en iterable y usamos el método `next`

In [ ]:
criticas_iter = iter(criticas)
print(next(criticas_iter))

Al contrario que el objeto generado con la función `generaCriticas` el objeto de `PreprocesaCriticas` se reinicia cada vez que lo convertimos en *iterable*:

Calculamos los vectores de palabras de todo el corpus con el modelo `word2vec` que acepta como argumento de entrada un objeto de tipo `iterator` como el creado

In [ ]:
#Cálculo de los vectores de palabras
from gensim.models import Word2Vec

model = Word2Vec(criticas, #iterador con los documentos
                               size=10,          #tamaño del vector
                               window=5,         #nº de términos adyacentes que usamos para el cálculo
                               min_count=5,      #nº mínimo de apariciones del término para contarlo
                               iter=100
                              )

#una vez entrenado el modelo nos quedamos con los vectores calculados
#si no se van a actualizar los vectores con nuevos documentos
model = model.wv
len(model.vocab)

Seleccinamos aleatoriamente 25 palabras del conjunto calculado

In [ ]:
import numpy as np

palabras_sample = np.random.choice(model.index2word, 25, replace=False)

In [ ]:
palabras_sample

### Ejercicio 5
Comprueba cómo funciona el modelo buscando las palabras más similares semánticamente a "trama", "peli" y "película"

## Extracción de características *sparse* con `gensim`
Vamos a generar las matrices TF-IDF y BoW del corpus con la librería Gensim mediante *data streming* para no tener que cargar en memoria los textos.